In [ ]:
# Downloading modules 
!pip install simpletransformers  # force install simpletrransformers in colab-environment

# Importing modules
import pandas as pd
import simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs

import sklearn
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, KFold

import torch
from torch.nn.modules.activation import Threshold

import os
import numpy as np
import wandb
import logging

In [3]:
!wandb login  # using this requires a Weights & Biases account: https://wandb.ai/login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
# mount colab to Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# importing data
data = pd.read_csv('/content/drive/MyDrive/ModelFolder/data_eigstig_text_label.csv')  # must be the exact path of the data

In [6]:
# set logging information 
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Define hyperparameter sweep-values
sweep_configuration = {
    'method': 'random',
    'metric': {'name': 'eval_loss', 'goal': 'maximize'},
    'parameters': 
    {
        'batch_size': {'values': [8, 32, 64]},
        'epochs': {'values': [5, 20, 50]},
        'lr': {'values': [0.01, 0.001, 0.0001]}     }
}

# refer sweep to wandb project
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Parameter_sweep')

# split data into train and test
train, test = train_test_split(data, test_size=0.1)

# train data to use for training and validation (splitting later on)
train_data = train
eval_data =  test



# Define model arguments
model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.output_dir = '/content/drive/MyDrive/NLP_exam/sweep/BERT_ASD_classification_model'
#model_args.manual_seed = 4  # comment out if cross-validating
model_args.use_multiprocessing = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 8
model_args.save_best_model = True
model_args.wandb_project = "Parameter_sweep"

def training_model():
  ''' 
  This function initializes a Weights & Biases project called "Parameter_sweep"
  It trains models with different combinations of hyperparameters with 5 folds for cross-validation. 
  Model arguments are set for a "bert-base-uncased" model that uses GPU.
  The model performs training on a training set and appends results to a list called "results"
  '''
    
  # set k_fold-specifics
  n =  5
  seed = 43
  kf = KFold(n_splits=n, random_state = seed, shuffle=True)

  # Defining Model using k-folds
  results = []
  val_train = []
  for train_index, val_index in kf.split(train_data):
      # Initialize a new wandb run
      wandb.init() 
      # splitting Dataframe (dataset not included)
      train_df = train_data.iloc[train_index]
      val_df = train_data.iloc[val_index]
      # Defining Model
      model = ClassificationModel('bert', 'bert-base-uncased', use_cuda=True, args=model_args, num_labels = 2, weight=[0.4, 0.6], 
                                    sweep_config=wandb.config)
      # train the model
      model.train_model(train_df, eval_df = val_df)
      # validate the model
      result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=accuracy_score) # , acc = accuracy_score
      print(result['acc'])
      # append model score
      results.append(result['acc'])

      # log validation loss
      wandb.log({"train-loss": wandb.summary['train_loss']})

      # sync wandb
      wandb.join()
        

# refer the model to the wandb id
wandb.agent(sweep_id, training_model)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: asrxglel
Sweep URL: https://wandb.ai/noget/Parameter_sweep/sweeps/asrxglel


wandb: Agent Starting Run: 9a047pjc with config:
wandb: 	batch_size: 64
wandb: 	epochs: 20
wandb: 	lr: 0.01
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kmn1998 (noget). Use `wandb login --relogin` to force relogin


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/3505 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/220 [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.
